<a href="https://colab.research.google.com/github/Prathika/ML_online_order_place_prediction/blob/master/online_order_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving training_data.csv to training_data.csv


In [0]:
import pandas as pd
import io
train_data = pd.read_csv(io.BytesIO(uploaded['training_data.csv']))

In [0]:
dummies = pd.get_dummies(train_data.funnel_level)

In [0]:
train_data_dummies= pd.concat([train_data,dummies],axis='columns')
train_data_dummies.drop('funnel_level',axis='columns',inplace=True)

In [0]:
def trim_prefix_link(data):
    return int(data.split('link')[1])
train_data_dummies['link'] = train_data_dummies['link'].apply(trim_prefix_link)

In [0]:
train_data_dummies.ad.fillna(0, inplace=True)
train_data_dummies.ad.isnull().sum()

0

In [0]:
from datetime import datetime
train_data_dummies['duration'] = train_data_dummies['timestamp'].apply(lambda x: datetime.strptime(str(x)[0:19],'%Y-%m-%dT%H:%M:%S'));
train_data_dummies.sort_values(['id','duration'],inplace=True)

In [0]:
train_data_dummies.head()

,id,ad,link,timestamp,checkout,order_placed,grp,lower,middle,upper,duration
153915,session0,0,1357209665544,2018-04-30T02:00:56.000-04:00,0,0,grp936302870528,0,0,1,2018-04-30 02:00:56
133158,session0,0,1125281431562,2018-04-30T02:01:52.000-04:00,0,0,grp936302870528,0,0,1,2018-04-30 02:01:52
3335,session0,0,1288490188815,2018-04-30T02:01:59.000-04:00,0,0,grp1340029796352,0,0,1,2018-04-30 02:01:59
61277,session0,0,1468878815250,2018-04-30T02:02:19.000-04:00,0,0,grp1340029796352,0,0,1,2018-04-30 02:02:19
172876,session0,ad481036337152,472446402562,2018-04-30T02:07:05.000-04:00,0,0,grp309237645312,0,1,0,2018-04-30 02:07:05


In [0]:
def time_to_minutes(time_str):  
    try:  
      
        time_str = str(time_str.split('days ')[1])
      
        hours, minutes, seconds = time_str.split(':')  
        seconds = seconds.split("\n")[0]
        return int(hours)*60 + int(minutes) + int(seconds)/60.0  
        
    except ValueError:  
        return -1  
  
    
 
    

In [0]:
def get_duration(record):
  try:
    diff = train_data_dummies.loc[train_data_dummies['id'] == record]['duration'].max() - train_data_dummies.loc[train_data_dummies['id'] == record]['duration'].min()
  except Exception as e:
    diff = -1
    return diff

In [0]:
train_data_dummies['dur'] = train_data_dummies.groupby('id')['duration'].transform(lambda x:\
                                                                                     str(x.max()-x.min()))


In [0]:
train_data_dummies['total_duration'] = train_data_dummies.groupby('id')['dur'].transform(lambda x:time_to_minutes(str(x)))

In [0]:
train_data_dummies.describe(include='all')

,id,ad,link,timestamp,checkout,order_placed,grp,lower,middle,upper,duration,dur
count,199410,199410.0,1.994100e+05,199410,199410.000000,199410.000000,199410,199410.000000,199410.000000,199410.000000,199410,199410
unique,14014,10389.0,NaN,62697,NaN,NaN,108,NaN,NaN,NaN,62697,3482
top,session51539607562,0.0,NaN,2018-04-30T16:18:24.000-04:00,NaN,NaN,grp1340029796352,NaN,NaN,NaN,2018-04-30 16:18:24,0 days 00:00:23
freq,235,79375.0,NaN,19,NaN,NaN,93084,NaN,NaN,NaN,19,2924
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-30 00:00:00,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-30 23:59:53,NaN
mean,NaN,NaN,8.515218e+11,NaN,0.416529,0.198155,NaN,0.102081,0.292272,0.605647,NaN,NaN
std,NaN,NaN,4.596463e+11,NaN,0.492985,0.398610,NaN,0.302756,0.454808,0.488713,NaN,NaN
min,NaN,NaN,0.000000e+00,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,5.239860e+11,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN


In [0]:
# Total duration calcuated so drop the timestamp
train_data_dummies.drop('timestamp',axis='columns',inplace=True)

In [0]:
train_data_dummies.drop('duration',axis='columns',inplace=True)

In [0]:
train_data_dummies

,id,ad,link,checkout,order_placed,grp,lower,middle,upper,dur,total_duration
153915,session0,0,1357209665544,0,0,grp936302870528,0,0,1,0 days 08:46:02,526.033333
133158,session0,0,1125281431562,0,0,grp936302870528,0,0,1,0 days 08:46:02,526.033333
3335,session0,0,1288490188815,0,0,grp1340029796352,0,0,1,0 days 08:46:02,526.033333
61277,session0,0,1468878815250,0,0,grp1340029796352,0,0,1,0 days 08:46:02,526.033333
172876,session0,ad481036337152,472446402562,0,0,grp309237645312,0,1,0,0 days 08:46:02,526.033333
75394,session0,ad481036337152,1030792151074,0,0,grp1589137899520,0,1,0,0 days 08:46:02,526.033333
75395,session0,ad481036337152,1030792151074,0,0,grp1589137899520,0,1,0,0 days 08:46:02,526.033333
79183,session0,ad481036337152,781684047909,0,0,grp1589137899520,0,1,0,0 days 08:46:02,526.033333
187206,session0,ad481036337152,910533066768,0,0,grp1314259992576,0,1,0,0 days 08:46:02,526.033333
30741,session0,ad481036337152,532575944721,0,0,grp1279900254208,0,1,0,0 days 08:46:02,526.033333


In [0]:
#train_data_dummies['dd1'] = train_data_dummies.groupby("duration").my_label.agg(get_count)
#train_data_dummies

In [0]:
# EXAMPLE
#df = pd.DataFrame({'Date': ['2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05', '2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05'], 'Sym': ['aapl', 'aapl', 'aapl', 'aapl', 'aaww', 'aaww', 'aaww', 'aaww'], 'Data2': [11, 8, 10, 15, 110, 60, 100, 40],'Data3': [5, 8, 6, 1, 50, 100, 60, 120]})
#df['Data4'] = df['Data3'].groupby(df['Date']).transform()
#df